![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## plants-phidgets-data

This is a utility notebook, to collect data from sensors around a plant and store in an online spreadsheet. The spreadsheet can then be accessed by anyone with the web address, to analysize the data.

Ask M. Lamoureux for details. May 2023.

<h2 style="color: red">IMPORTANT</h2>
You MUST click the <span style="color:red">BIG, RED DISCONNECT BUTTON</span> at the end of this notebook when you are done. This tells the software to release the Phidget device, so it can be used by other notebooks you might try later.

## Overview

It takes quite a bit of setup to use this notebook appropriately.

- you need the Phidget VINT device, with three sensors attached
    - the temperature/humidity sensor (device type HUM1001_01)
    - the light sensor (device type LUX1000_0)
    - the moisture sensor (device type HUM1100)
- a computer (Mac, Windows, Raspberry Pi) with USB connection and internet connection
- the Chrome or Chromium Web browser
- ideally, Jupyter software installed on the computer, to run independently of the Callysto Hub

You should also choose the name of your online spreadsheet. Here, we are using the webpage called https://ethercalc.net/MyWeirdName23

This what a typical setup looks like:

<img src="images/plant2.jpg" alt="A plant with sensor" width="400"/>
<div align="center">
A basement window with our plant.
</div>

The notebook runs both Python code and Javascript (JS) code. The JS code is the front end, that talks to the browser and the Phidget devices collecting data. The Python code is the back end, taking care of things like dates, data format, and saving the data onto the online web spreadsheet.

One very tricky aspect is getting the JS and Python engines talking to each other. This makes use of the "Comms" package in Jupyter that manages the communication between these two engines. One key feature of the Python engine (or kernel) is that when it is executing code, it cannot be interrupted by the JS engine. So even if you have a simple loop in Python, or a sleep command to pause Python, the JS engine cannot interrupt it to give it new data.

So, our trick is to set a timer in JS that gathers data every few minutes, then sends a message to Python saying "deal with this data." This triggers a Python code that looks at the data and decides if it is time to save it to the web. So in fact, the Python is not "doing" anything most of the time, so JS is not actually interrupting it. Instead, it is telling Python "now is the time to deal with this data" and the Python engine kicks in. 

### Spreadsheet

The spreadsheet can be viewed here:

https://ethercalc.net/MyWeirdName23

The 4 numbers following the date and time are temperature, humidity, moisture in the soil, and the light level. Here is a sample.

|Date	|Time	|Temperature	|Humidity	|Moisture	|Luminance |
| ------ | ------ | ------ | ------ | ------ | ------ | 
|2023-04-30 |21:43:42	|20.64	|50.29	|0.199	|0.0193 |
|2023-04-30	|21:43:43	|20.64	|50.29	|0.199	|0.0193 |
|2023-04-30	|21:43:47	|20.66	|50.26	|0.198	|0.044 |
|2023-04-30	|21:43:47	|20.64	|50.26	|0.198	|0.044 |
|2023-04-30	|21:43:52	|20.66	|50.24	|0.199	|0.043 |

## Setup the Phidget devices

You must be running on Chrome or Chromium on your laptop or desktop, and the Phidget VINT device needs to be connected to the USB port on your computer. 

The VINT device must have three sensors attached: the temperature/humidity sensor, the light sensor, and the soil moisture sensor.

The main steps are:

0. Import some Python libraries
1. Import the Phidget library, in Javascript
2. Define and open the USB connection
3. Open the Phidget devices
4. Set up Python code to post data to the online web spreadsheet
5. Set up the Comms channel, so JS and Python can talk to each other
6. Initiate the JS engine to start collecting data
7. Let it run for a few hours, or days, to collect data
8. When we are done, close the devices and USB connection


## Step 0. Python Libraries

Let's import a few Python libraries that we need. 

In [1]:
import ipywidgets as widgets
from time import sleep
import requests
from datetime import datetime, timedelta

## Step 1. Phidgets library.

Now import the Phidgets library. 

We are using a special library for debugging purposes. We will need to update this when the Phidgets company finishes their testing. 

Special library is here: https://www.phidgets.com/downloads/testing/phidget22

In [2]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://www.phidgets.com/downloads/testing/phidget22'], 
    },                                         
});
require(['phidget22'], (phidget22) => {
   window.phidget22 = phidget22; 
});

<IPython.core.display.Javascript object>

In [3]:
## We pause for a second here, to allow the library to load
sleep(1)

## Step 2. Open USB

We now open a USB connection.  

In [4]:
%%js

if (window.usbconn === undefined) {
    
    element.text("Creating a new USB Connection.");
    
    window.usbconn = new phidget22.USBConnection();
    
    usbconn.connect().then(() => {
        usbconn.requestWebUSBDeviceAccess();
    }).catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
}

<IPython.core.display.Javascript object>

In [5]:
## We rest for a bit while the USB connects
sleep(1)

### Confirm the connection 

In [6]:
%%js
element.text(usbconn.connected);

<IPython.core.display.Javascript object>

## Step 3. Connect the sensors

We make a request to open the four different sensors. We pause between each one, and check that they are connected. 

In [7]:
%%js
window.humSensor = new phidget22.HumiditySensor();
humSensor.open();

<IPython.core.display.Javascript object>

In [8]:
sleep(1)

In [9]:
%%js
element.text("Is the humidity sensor attached? " + humSensor.attached);

<IPython.core.display.Javascript object>

In [10]:
%%js
window.tempSensor = new phidget22.TemperatureSensor();
tempSensor.open();

<IPython.core.display.Javascript object>

In [11]:
sleep(1)

In [12]:
%%js
element.text("Is the temperature sensor attached? " + tempSensor.attached);

<IPython.core.display.Javascript object>

In [13]:
%%js
window.vrSensor = new phidget22.VoltageRatioInput();
vrSensor.open();

<IPython.core.display.Javascript object>

In [14]:
sleep(1)

In [15]:
%%js
element.text("Is the moisture sensor attached? " + vrSensor.attached);

<IPython.core.display.Javascript object>

In [16]:
%%js
window.liteSensor = new phidget22.LightSensor();
liteSensor.open();

<IPython.core.display.Javascript object>

In [17]:
sleep(1)

In [18]:
%%js
element.text("Is the light sensor attached? " + liteSensor.attached);

<IPython.core.display.Javascript object>